In [ ]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("yye090_722").getOrCreate()


In [ ]:
df = spark.read.load("./Datasets/*csv",format ="csv",header="true")



In [ ]:
df = df.withColumnRenamed("PM2.5","PM25")
df = df.drop(*["No","PM10","PRES","DEWP","RAIN"])
df = df.na.drop()
df = df.filter((df.PM25 != 'NA') & (df.SO2 != 'NA') & (df.NO2 != 'NA') &\
               (df.CO != 'NA') &(df.O3 != 'NA') &(df.TEMP != 'NA') &(df.wd != 'NA') &(df.WSPM != 'NA'))


In [ ]:
df = df.na.drop()
df = df.filter((df.PM25 != 'NA') & (df.SO2 != 'NA') & (df.NO2 != 'NA') &\
               (df.CO != 'NA') &(df.O3 != 'NA') &(df.TEMP != 'NA') &(df.wd != 'NA') &(df.WSPM != 'NA'))


In [ ]:
from pyspark.sql import *
df.printSchema()
changedTypedf = df.withColumn("year", df["year"].cast("integer")).withColumn("month", df["month"].cast("integer"))\
.withColumn("hour", df["hour"].cast("integer"))\
.withColumn("day", df["day"].cast("integer"))\
.withColumn("PM25", df["PM25"].cast("float"))\
.withColumn("SO2", df["SO2"].cast("float"))\
.withColumn("NO2", df["NO2"].cast("float"))\
.withColumn("CO", df["CO"].cast("float"))\
.withColumn("O3", df["O3"].cast("float"))\
.withColumn("TEMP", df["TEMP"].cast("float"))\
.withColumn("WSPM", df["WSPM"].cast("float"))
changedTypedf.printSchema()


In [ ]:
cols = ["PM25","SO2","NO2","CO","O3","TEMP"]
bounds ={}
for col in cols:
    quantiles = changedTypedf.approxQuantile(col,[0.25,0.75],0)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5*IQR,quantiles[1] +1.5*IQR]



In [ ]:
outliers =  changedTypedf.select(*["year",'month','day','hour',"WSPM","wd","station"]+[
    (
            (changedTypedf[c]<bounds[c][0])|
            (changedTypedf[c]>bounds[c][1])
    ).alias(c + "_outliers") for c in cols
    ])

In [ ]:
new_df1 = changedTypedf.join(outliers,on=["year","month","day","hour","WSPM","wd","station"])
new_df1 = new_df1.filter((new_df1.PM25_outliers == False) & (new_df1.SO2_outliers == False) & (new_df1.SO2_outliers == False) &\
                       (new_df1.NO2_outliers == False) &\
                       (new_df1.CO_outliers == False) &\
                       (new_df1.O3_outliers == False) &\
                       (new_df1.TEMP_outliers == False))
new_df1= new_df1.drop(*["WPSM",'wd','PM25_outliers',\
 'SO2_outliers',\
 'NO2_outliers',\
 'CO_outliers',\
 'O3_outliers',\
 'TEMP_outliers'])


In [ ]:
from pyspark.sql import functions as F
df2 = new_df1.withColumn("Season",\
    F.when(F.col("month").between(3,5),"Spring")\
    .when(F.col("month").between(6,8),"Summer")\
    .when(F.col("month").between(9,11),"Autumn")\
    .otherwise("Winter")\
)
df2 = df2.withColumn("PM_Rating",\
    F.when(F.col("PM25").between(0,12),"Excellent")\
    .when(F.col("PM25").between(12,35.4),"Moderate")\
    .when(F.col("PM25").between(35.4,55.4),"Unhealthy for Sensitive Groups")\
    .when(F.col("PM25").between(55.4,150.4),"Unhealthy")\
    .when(F.col("PM25").between(150.4,250.4),"Very Unhealthy")\
    .when(F.col("PM25").between(250.4,500.4),"Hazardous")\
    .otherwise("Severe")\
)
df2.show()

In [ ]:
df2 = df2.filter((df2.year != 2016))
df2 = df2.filter((df2.year != 2017))

In [ ]:
df2.printSchema()

In [ ]:
df2.write.csv("./final1.csv",header="true")